In [1]:
import ray
from typing import Dict, List
import numpy as np

ds = ray.data.read_json('../etl_out/partition_by_date')

2024-03-10 00:06:20,184	INFO worker.py:1724 -- Started a local Ray instance.


In [11]:
ds_10 = ds.limit(10)

In [20]:
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import AutoModel
from llama_index.core.node_parser import  SentenceSplitter #SimpleNodeParser
from llama_index.core.schema import Node
from llama_index.core.schema import Document
import pandas as pd
import pyarrow as pa

def convert_documents_into_nodes(data: Dict[str, np.ndarray]) -> List[Dict[str, Node]]:
    all_text = data['title'] + data['content']
    documents = [ Document(text=t) for t in all_text]
    nodes = SentenceSplitter.get_nodes_from_documents([documents])
    return [{"node": node} for node in nodes]

class EmbedNodes:
    def __init__(self):
        import os
        os.environ['HTTPS_PROXY'] = 'http://100.109.83.118:808/'
        self.embedding_model = HuggingFaceEmbeddings(model_name='jinaai/jina-embeddings-v2-base-zh',model_kwargs= {"trust_remote_code":"True"})

    #列存储的方式
    def __call__(self, node_batch: Dict[str, List[Node]]) -> Dict[str, List[Node]]:
       
        nodes = node_batch["node"]
        text = [node.text for node in nodes]
        embeddings = self.embedding_model.embed_documents(text)
        assert len(nodes) == len(embeddings)

        for node, embedding in zip(nodes, embeddings):
            node.embedding = embedding
        return {"embedded_nodes": nodes}

In [21]:
nodes = ds.flat_map(convert_documents_into_nodes)


In [22]:
embeds = nodes.map_batches(EmbedNodes, concurrency=1)

In [23]:
embeds.write_parquet('./test.parquet')

2024-03-10 00:17:32,870	INFO set_read_parallelism.py:115 -- Using autodetected parallelism=200 for stage ReadJSON to satisfy DataContext.get_current().min_parallelism=200.
2024-03-10 00:17:32,872	INFO streaming_executor.py:112 -- Executing DAG InputDataBuffer[Input] -> ActorPoolMapOperator[ReadJSON->FlatMap(convert_documents_into_nodes)->MapBatches(EmbedNodes)] -> TaskPoolMapOperator[Write]
2024-03-10 00:17:32,873	INFO streaming_executor.py:113 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), exclude_resources=ExecutionResources(cpu=0, gpu=0, object_store_memory=0), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
2024-03-10 00:17:32,875	INFO streaming_executor.py:115 -- Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().execution_options.verbose_progress = True`


2024-03-10 00:17:33,689	INFO actor_pool_map_operator.py:114 -- ReadJSON->FlatMap(convert_documents_into_nodes)->MapBatches(EmbedNodes): Waiting for 1 pool actors to start...


Running 0:   0%|          | 0/200 [00:00<?, ?it/s]

2024-03-10 00:17:54,966	ERROR streaming_executor_state.py:496 -- An exception was raised from a task of operator "ReadJSON->FlatMap(convert_documents_into_nodes)->MapBatches(EmbedNodes)". Dataset execution will now abort. To ignore this exception and continue, set DataContext.max_errored_blocks.
2024-03-10 00:17:54,979	WARNING actor_pool_map_operator.py:278 -- To ensure full parallelization across an actor pool of size 1, the Dataset should consist of at least 1 distinct blocks. Consider increasing the parallelism when creating the Dataset.


RayTaskError(TypeError): [36mray::ReadJSON->FlatMap(convert_documents_into_nodes)->MapBatches(EmbedNodes)()[39m (pid=107029, ip=192.168.1.35, actor_id=7c9ef0773aaf52b09f8acf8d01000000, repr=MapWorker(ReadJSON->FlatMap(convert_documents_into_nodes)->MapBatches(EmbedNodes)))
    yield from _map_task(
  File "/home/userroot/miniconda3/envs/rag/lib/python3.10/site-packages/ray/data/_internal/execution/operators/map_operator.py", line 418, in _map_task
    for b_out in map_transformer.apply_transform(iter(blocks), ctx):
  File "/home/userroot/miniconda3/envs/rag/lib/python3.10/site-packages/ray/data/_internal/execution/operators/map_transformer.py", line 139, in apply_transform
    iter = transform_fn(iter, ctx)
  File "/home/userroot/miniconda3/envs/rag/lib/python3.10/site-packages/ray/data/_internal/execution/operators/map_transformer.py", line 288, in __call__
    first = next(block_iter, None)
  File "/home/userroot/miniconda3/envs/rag/lib/python3.10/site-packages/ray/data/_internal/e

In [23]:
print('hello')

for a, b in c:
    print(a)
#a = [2, 5]
print(list(c))

hello
1
4
[]
